In [37]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense
import altair as alt
import json
from vega_datasets import data
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [38]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [39]:
# read data
df = pd.read_csv('chemical_inventory_usage.csv')

In [40]:
# transfer id to date 
df['date'] = pd.to_datetime(df['id'], unit='D', origin=pd.Timestamp('2020-01-01'))

In [41]:
# data
df_agg = df.groupby(['date', 'location', 'reason', 'chemical', 'unit']).agg(amount_taken=('amount_taken', 'sum')).reset_index()

In [42]:
# encode location
location_encoder = OneHotEncoder(sparse_output=False)
location_encoded = location_encoder.fit_transform(df_agg[['location']])
location_encoded_df = pd.DataFrame(location_encoded, columns=location_encoder.get_feature_names_out(['location']))
df_agg = pd.concat([df_agg, location_encoded_df], axis=1)


In [43]:
# encode reason
reason_encoder = OneHotEncoder(sparse_output=False)
reason_encoded = reason_encoder.fit_transform(df_agg[['reason']])
reason_encoded_df = pd.DataFrame(reason_encoded, columns=reason_encoder.get_feature_names_out(['reason']))
df_agg = pd.concat([df_agg, reason_encoded_df], axis=1)

In [44]:
# get the chemical names
chemicals = df_agg['chemical'].unique()

In [45]:
# save all the chemicals predictions and evaluations 
predictions = {}
evaluations = {}

In [46]:
# modify and predict 
for chemical in chemicals:
    # 筛选出当前化学品的数据
    df_chemical = df_agg[df_agg['chemical'] == chemical].copy()

    # 在这里对 location 和 reason 列进行独热编码
    df_chemical = pd.get_dummies(df_chemical, columns=['location', 'reason'])

    # 构建滞后特征，这里以7天为例
    for i in range(1, 8):
        df_chemical[f'amount_taken_lag_{i}'] = df_chemical.groupby('chemical')['amount_taken'].shift(i)
    df_chemical = df_chemical.dropna()

    # 划分训练集和测试集
    train_size = int(len(df_chemical) * 0.8)
    train, test = df_chemical.iloc[:train_size], df_chemical.iloc[train_size:]

    # 定义特征和目标变量
    X_train = train.drop(columns=['date', 'amount_taken', 'unit', 'chemical'])  # 不再需要删除 location 和 reason
    y_train = train['amount_taken']
    X_test = test.drop(columns=['date', 'amount_taken', 'unit', 'chemical'])  # 不再需要删除 location 和 reason
    y_test = test['amount_taken']

    # 确保训练集和测试集的列一致 (因为独热编码可能导致列数不同)
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    # 归一化特征
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)


    # LSTM model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # 将特征转换为LSTM输入格式
    X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
    X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

    # train the model
    model.fit(X_train_scaled, y_train, epochs=20, batch_size=32)

    # prediction
    y_pred = model.predict(X_test_scaled)

    # 1. 创建一个与 X_test 形状相同的全零矩阵
    y_pred_full = np.zeros(X_test.shape)  

    # 2. 将预测值填充到最后一列
    y_pred_full[:, -1] = y_pred.flatten()

    # 3. 逆归一化
    y_pred = scaler.inverse_transform(y_pred_full)[:, -1]  

    # 保存预测结果（这里也需要调整，因为y_pred现在是一维数组）
    predictions[chemical] = pd.DataFrame({'date': test['date'], 'actual': y_test, 'predicted': y_pred})
    
        # evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    # caculate MAPE
    epsilon = 1e-5  # 防止除零错误
    mape = np.mean(np.abs((y_test - y_pred) / (y_test + epsilon))) * 100
    
    # caculate R-squared
    r2 = r2_score(y_test, y_pred)

    evaluations[chemical] = {
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape,
        "R-squared":r2
    }


Epoch 1/20
6/6 [==============================] - 1s 15ms/step - loss: 85054.4922
Epoch 2/20
6/6 [==============================] - 0s 13ms/step - loss: 85026.3203
Epoch 3/20
6/6 [==============================] - 0s 12ms/step - loss: 84993.9141
Epoch 4/20
6/6 [==============================] - 0s 13ms/step - loss: 84949.5703
Epoch 5/20
6/6 [==============================] - 0s 13ms/step - loss: 84873.4766
Epoch 6/20
6/6 [==============================] - 0s 12ms/step - loss: 84618.3594
Epoch 7/20
6/6 [==============================] - 0s 13ms/step - loss: 144096.0625
Epoch 8/20
6/6 [==============================] - 0s 12ms/step - loss: 83369.2656
Epoch 9/20
6/6 [==============================] - 0s 14ms/step - loss: 84552.7344
Epoch 10/20
6/6 [==============================] - 0s 12ms/step - loss: 84644.0312
Epoch 11/20
6/6 [==============================] - 0s 13ms/step - loss: 84676.3359
Epoch 12/20
6/6 [==============================] - 0s 11ms/step - loss: 84690.4844
Epoch 13/20


In [47]:
for chemical, result in predictions.items():
    print(f"\nPrediction results for {chemical}:")
    print(result.head().to_markdown(index=False, numalign="left", stralign="left"))


Prediction results for Chemical_D:
| date                | actual   | predicted   |
|:--------------------|:---------|:------------|
| 2022-03-23 00:00:00 | 419.61   | 406.776     |
| 2022-03-28 00:00:00 | 296.046  | 406.339     |
| 2022-04-01 00:00:00 | 401.624  | 415.834     |
| 2022-04-03 00:00:00 | 167.416  | 421.625     |
| 2022-04-09 00:00:00 | 369.013  | 428.316     |

Prediction results for Chemical_A:
| date                | actual   | predicted   |
|:--------------------|:---------|:------------|
| 2022-04-15 00:00:00 | 249.686  | 352.303     |
| 2022-04-17 00:00:00 | 457.508  | 331.206     |
| 2022-04-19 00:00:00 | 290.714  | 355.616     |
| 2022-04-21 00:00:00 | 7.53413  | 346.474     |
| 2022-04-27 00:00:00 | 43.5895  | 346.47      |

Prediction results for Chemical_C:
| date                | actual   | predicted   |
|:--------------------|:---------|:------------|
| 2022-02-11 00:00:00 | 423.38   | 506.187     |
| 2022-02-14 00:00:00 | 398.85   | 545.322     |
| 2022-02-

In [48]:
    # 输出每个化学品的评估结果
    for chemical, metrics in evaluations.items():
        print(f"\nEvaluation metrics for {chemical}:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")


Evaluation metrics for Chemical_D:
MSE: 44396.7296
MAE: 171.9720
RMSE: 210.7053
MAPE: 377.2534
R-squared: -1.3169

Evaluation metrics for Chemical_A:
MSE: 36698.0643
MAE: 167.8056
RMSE: 191.5674
MAPE: 399.7551
R-squared: -0.6189

Evaluation metrics for Chemical_C:
MSE: 95180.4645
MAE: 278.0679
RMSE: 308.5133
MAPE: 513.2944
R-squared: -4.5183

Evaluation metrics for Chemical_B:
MSE: 21392.4061
MAE: 120.4447
RMSE: 146.2614
MAPE: 230.6673
R-squared: -0.1023
